In [19]:
import pandas as pd

In [20]:
reddit = pd.read_csv("datasets/upvotes.csv")
reddit.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,52664,a,3942.0,2.0,155623,7855.0,42.0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0
2,468453,c,1358.0,4.0,56177,8067.0,60.0
3,96996,a,264.0,3.0,168793,27064.0,9.0
4,131465,c,4271.0,4.0,112223,13986.0,83.0


In [21]:
reddit.shape

(330045, 7)

In [22]:
reddit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330045 entries, 0 to 330044
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ID          330045 non-null  int64  
 1   Tag         330045 non-null  object 
 2   Reputation  330045 non-null  float64
 3   Answers     330045 non-null  float64
 4   Username    330045 non-null  int64  
 5   Views       330045 non-null  float64
 6   Upvotes     330045 non-null  float64
dtypes: float64(4), int64(2), object(1)
memory usage: 17.6+ MB


In [23]:
reddit = reddit.dropna()

In [24]:
reddit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330045 entries, 0 to 330044
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ID          330045 non-null  int64  
 1   Tag         330045 non-null  object 
 2   Reputation  330045 non-null  float64
 3   Answers     330045 non-null  float64
 4   Username    330045 non-null  int64  
 5   Views       330045 non-null  float64
 6   Upvotes     330045 non-null  float64
dtypes: float64(4), int64(2), object(1)
memory usage: 20.1+ MB


In [25]:
reddit['Tag'].value_counts()

c    72458
j    72232
p    43407
i    32400
a    31695
s    23323
h    20564
o    14546
r    12442
x     6978
Name: Tag, dtype: int64

### Going to one hot encode the above values

In [26]:
from sklearn.preprocessing import OneHotEncoder
ohEncoder = OneHotEncoder()
tag_cat = reddit[["Tag"]]
tags = ohEncoder.fit_transform(tag_cat)
tags

<330045x10 sparse matrix of type '<class 'numpy.float64'>'
	with 330045 stored elements in Compressed Sparse Row format>

In [27]:
tags.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
ohEncoder.categories_

[array(['a', 'c', 'h', 'i', 'j', 'o', 'p', 'r', 's', 'x'], dtype=object)]

### transforming to add new column that shows the views ratioed by the reputation

In [29]:
import sklearn
import numpy as np
# Going to use the apply function similar to the transform function

#make sure no reputation values are zero so division by zero will not happen
reddit = reddit[reddit.Reputation != 0]
reddit['ViewsPerPop'] = reddit.apply(lambda row : row.Views / row.Reputation, axis = 1)
reddit.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes,ViewsPerPop
0,52664,a,3942.0,2.0,155623,7855.0,42.0,1.992643
1,327662,a,26046.0,12.0,21781,55801.0,1175.0,2.142402
2,468453,c,1358.0,4.0,56177,8067.0,60.0,5.940353
3,96996,a,264.0,3.0,168793,27064.0,9.0,102.515152
4,131465,c,4271.0,4.0,112223,13986.0,83.0,3.274643


### using ensemble methods to get a predictor for upvotes

[source](https://www.geeksforgeeks.org/ensemble-methods-in-python/)

In [69]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
#
# # importing machine learning models for prediction
# from sklearn.ensemble import RandomForestRegressor
# import xgboost as xgb
# from sklearn.linear_model import LinearRegression
#
# # loading train data set in dataframe from train_data.csv file
# df = pd.read_csv("train_data.csv")
#
# # getting target data from the dataframe
# target = df["target"]
#
# # getting train data from the dataframe
# train = df.drop("target")
#
# # Splitting between train data into training and validation dataset
# X_train, X_test, y_train, y_test = train_test_split(
#     train, target, test_size=0.20)
#
# # initializing all the model objects with default parameters
# model_1 = LinearRegression()
# model_2 = xgb.XGBRegressor()
# model_3 = RandomForestRegressor()
#
# # training all the model on the training dataset
# model_1.fit(X_train, y_target)
# model_2.fit(X_train, y_target)
# model_3.fit(X_train, y_target)
#
# # predicting the output on the validation dataset
# pred_1 = model_1.predict(X_test)
# pred_2 = model_2.predict(X_test)
# pred_3 = model_3.predict(X_test)
#
# # final prediction after averaging on the prediction of all 3 models
# pred_final = (pred_1+pred_2+pred_3)/3.0
#
# # printing the root mean squared error between real value and predicted value
# print(mean_squared_error(y_test, pred_final))

### simple start to prediction pipeline rand. forest
[followed this example](https://towardsdatascience.com/random-forest-in-python-24d0893d51c0)

edit: not too happy with the accuracy so lets attempt a pipeline for linear regression

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

reddit = pd.read_csv("datasets/upvotes.csv")

#realizing shape makes this take forever
# print(reddit.shape)
reddit = reddit[reddit.Upvotes != 0]
# reddit = reddit[reddit.Answers != 0]
# reddit = reddit[reddit.Reputation != 0]

reddit = reddit.head(10000)
print(reddit.shape)

reddit = reddit[reddit.Reputation != 0]
reddit['ViewsPerPop'] = reddit.apply(lambda row : row.Views / row.Reputation, axis = 1)

reddit = reddit.drop(['Username', 'ID', 'Answers'], axis=1)

reddit = pd.get_dummies(reddit)
upvotes = np.array(reddit['Upvotes'])

reddit = reddit.drop(['Upvotes'], axis=1)

reddit_list = list(reddit.columns)

reddit = np.array(reddit)

train_reddit, test_reddit, train_upvotes, test_upvotes = train_test_split(reddit, upvotes, test_size = 0.25, random_state = 42)

rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(train_reddit, train_upvotes)

# Use the forest's predict method on the test data
predictions = rf.predict(test_reddit)

# Calculate the absolute errors
errors = abs(predictions - test_upvotes)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_upvotes)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

(10000, 7)


### simple start to prediction pipeline linear
[followed this example (same guy)](https://towardsdatascience.com/simple-and-multiple-linear-regression-in-python-c928425168f9)
